# Data Download & Processing

The objective of this script is to create a common dataset for our analysis. It includes the following functions:
- download MTA bus data from May 1 to 31, 2017
- process data in each file 
- select only M15 and M15 SBS buses
- concatenate daily dataset into one consolidated dataset

---

# Download MTA Bus Data

- Download
- Unzip
- Remove zipped files

In [58]:
# Download Bus Time for all May, 2017
# Only do it at set up

'''
for i in range(1, 32):
    
    if i < 10: 
        file = 'bus_time_2017050' + str(i) + '.csv.xz'
    else: 
        file = 'bus_time_201705' + str(i) + '.csv.xz'
    
    url = 'http://data.mytransit.nyc.s3.amazonaws.com/bus_time/2017/2017-05/' + file 
    
    print("--- Downloading MTA Bus Data of May %i, 2017 ---" %i)
    !wget -nc $url -P './download'
    
    print("--- Unzipping MTA Bus Data of May %i, 2017 ---" %i)
    path = "./download/" + file
    !7za e $path -o./data
'''

'\nfor i in range(1, 32):\n    \n    if i < 10: \n        file = \'bus_time_2017050\' + str(i) + \'.csv.xz\'\n    else: \n        file = \'bus_time_201705\' + str(i) + \'.csv.xz\'\n    \n    url = \'http://data.mytransit.nyc.s3.amazonaws.com/bus_time/2017/2017-05/\' + file \n    \n    print("--- Downloading MTA Bus Data of May %i, 2017 ---" %i)\n    !wget -nc $url -P \'./download\'\n    \n    print("--- Unzipping MTA Bus Data of May %i, 2017 ---" %i)\n    path = "./download/" + file\n    !7za e $path -o./data\n'

In [59]:
'''file = 'http://data.mytransit.nyc.s3.amazonaws.com/gtfs/2015/gtfs_nyct_bus_20150103.zip'
!wget -nc $file -P './download'
'''

"file = 'http://data.mytransit.nyc.s3.amazonaws.com/gtfs/2015/gtfs_nyct_bus_20150103.zip'\n!wget -nc $file -P './download'\n"

In [60]:
'''

path = './download/gtfs_nyct_bus_20150103.zip'
!7za e $path -o./data

'''

"\n\npath = './download/gtfs_nyct_bus_20150103.zip'\n!7za e $path -o./data\n\n"

---

In [24]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.DataFrame()
total_df = pd.DataFrame()

In [45]:
trips = pd.read_csv("./data/trips.txt")
stops = pd.read_csv("./data/stops.txt")
schedules = pd.read_csv("./data/stop_times.txt")

### Process and Concatenate 3 Days of Bus Data for Testing

In [53]:
for i in range(1, 4):
    try:
        if i < 10: 
            file = './data/bus_time_2017050' + str(i) + '.csv'
        else: 
            file = './data/bus_time_201705' + str(i) + '.csv'

        # ingest into a data frame
        print(" --- Processing %s ---" %file)
        df = pd.read_csv(file)

        # convert timestamp to DateTime
        df['timestamp'] = pd.to_datetime(df['timestamp'], infer_datetime_format = True)
        df['Day'] = df['timestamp'].apply(lambda x: x.day)
        df['Hour'] =df['timestamp'].apply(lambda x: x.hour)
        df['DOW'] = df['timestamp'].dt.dayofweek

        days = {0:'Mon',1:'Tues',2:'Weds',3:'Thurs',4:'Fri',5:'Sat',6:'Sun'}
        df['DOW_Label'] = df['DOW'].apply(lambda x: days[x])
        
        print(" --- Done (1/3) ---")
        
        #throw away trips with <15 reports
        good_trips_only = df.groupby(df.trip_id).filter(lambda group: len(group) > 15)
        
        print(" --- Done (2/3) --- ")
        
        #concatenate 
    
        total_df = pd.concat([total_df, df])
        
        print(" --- Done (3/3) --- ")
        
    except Exception as ex:
        print(ex)
        pass

 --- Processing ./data/bus_time_20170501.csv ---


/opt/rh/anaconda/root/envs/PUI2016_Python3/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (9,10,11) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


 --- Done (1/3) ---
 --- Done (2/3) --- 
 --- Done (3/3) --- 
 --- Processing ./data/bus_time_20170502.csv ---
 --- Done (1/3) ---
 --- Done (2/3) --- 
 --- Done (3/3) --- 
 --- Processing ./data/bus_time_20170503.csv ---
 --- Done (1/3) ---
 --- Done (2/3) --- 
 --- Done (3/3) --- 


In [61]:
total_df.shape

(12661727, 16)

In [55]:
join_df = total_df.merge(trips, on='trip_id')

In [56]:
join_df.shape

(0, 21)

In [57]:
total_df.sample(10)

,DOW,DOW_Label,Day,Hour,bearing,block_assigned,dist_along_route,dist_from_stop,latitude,longitude,next_stop_id,progress,service_date,timestamp,trip_id,vehicle_id
247331,NaN,NaN,1,2,16.77,1,10856.32,176.94,40.735352,-73.865199,551310,0,20170430,2017-05-01 02:44:05,15377665-CPPB7-CP_B7-Sunday-10,630
623844,1.0,Tues,2,9,30.38,1,9324.49,89.99,40.761835,-73.866824,550934,0,20170502,2017-05-02 09:24:41,15370910-CPPB7-CP_B7-Weekday-10-SDon,649
1007725,2.0,Weds,3,11,115.12,1,587.77,97.43,40.710359,-73.870513,551067,0,20170503,2017-05-03 11:47:08,15387347-LGPB7-LG_B7-Weekday-10-SDon,3595
2691248,NaN,NaN,1,21,233.70,1,2809.96,177.24,40.754639,-73.980040,403257,0,20170501,2017-05-01 21:29:59,MA_B7-Weekday-SDon-102500_X30_607,2410
520752,1.0,Tues,2,6,230.27,1,8462.02,9.99,40.705276,-74.013779,404190,0,20170502,2017-05-02 06:01:04,YU_B7-Weekday-SDon-009000_X109_1,2670
2457513,2.0,Weds,3,20,341.31,1,1416.78,214.43,40.814124,-73.912526,101314,0,20170503,2017-05-03 20:02:12,WF_B7-Weekday-SDon-094500_BX17_123,7738
3051109,2.0,Weds,3,22,303.13,1,4334.39,502.23,40.684795,-73.792761,550052,0,20170503,2017-05-03 22:53:48,15355458-JKPB7-JK_B7-Weekday-10-SDon,3690
2243455,2.0,Weds,3,18,352.65,1,2870.98,301.48,40.626505,-74.160305,203537,0,20170503,2017-05-03 18:54:30,CA_B7-Weekday-SDon-087600_MISC_491,7003
2487567,NaN,NaN,1,20,201.20,1,567.44,5.08,40.704478,-73.797531,553159,0,20170501,2017-05-01 20:37:23,15355065-JKPB7-JK_B7-Weekday-10-SDon,4635
1774272,1.0,Tues,2,16,6.13,1,4495.32,14.35,40.599936,-73.946378,300206,0,20170502,2017-05-02 16:50:30,UP_B7-Weekday-SDon-074100_X2737_707,4870


In [62]:
trips.sample(10)

,route_id,service_id,trip_id,trip_headsign,direction_id,shape_id
63235,B9,JG_A5-Saturday,JG_A5-Saturday-115800_B9_25,BAY RDGE SHORE RD via AVENUE M via 60 ST,1,B90040
150771,M79,MQ_H5-Weekday,MQ_H5-Weekday-125100_M79_20,WEST SIDE-RIVERSIDE DR CROSSTOWN,1,M790026
132639,M23,MQ_A5-Sunday,MQ_A5-Sunday-079900_M23_15,EAST SIDE FDR DR CROSSTOWN,0,M230005
116424,M103,OH_A5-Saturday,OH_A5-Saturday-108700_M101_102,CITY HALL via LEX AV,1,M1030172
66534,BX11,WF_H5-Weekday,WF_H5-Weekday-146000_BX35_235,WASHINGTON HEIGHTS G W BRIDGE,1,BX110042
109491,BX8,WF_A5-Sunday,WF_A5-Sunday-129000_BX8_20,WESTCHESTER SQ,1,BX080082
175580,Q36,QV_A5-Weekday,QV_A5-Weekday-104500_Q36_5,JAMAICA 165 ST TERM,1,Q360148
150243,M79,MQ_A5-Weekday,MQ_A5-Weekday-127000_M79_18,YORKVILLE EAST END AV CROSSTOWN,0,M790024
198611,Q85,JA_A5-Weekday-SDon,JA_A5-Weekday-SDon-120700_MISC_567,JAMAICA PRSNS - ARCHR,1,Q851148
134690,M2,MV_J5-Weekday,MV_J5-Weekday-142000_M2_143,EAST VILLAGE 8 ST via 5 AV,1,M020171
